## sistemas de ecuaciones sobredeterminados

In [1]:
import numpy as np
import scipy.linalg as la

Resolvamos el sistema de ecuaciones

$$
\begin{align*}
x + 2y &= 3\\
3x+4y  &= 5
\end{align*}
$$

o, en forma matricial $A X = B$:

$$\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix} X = \begin{bmatrix}3 \\ 5 \end{bmatrix}
$$

In [2]:
a = np.array([[1,2]
             ,[3,4]])
b = np.array([3,5])

In [3]:
x = la.solve(a,b)

In [4]:
x

array([-1.,  2.])

In [5]:
a @ x

array([ 3.,  5.])

In [6]:
la.inv(a)

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

In [7]:
la.inv(a) @ b

array([-1.,  2.])

### múltiples lados derechos

Cuando hay múltiples lados derechos podemos ponerlos en una matriz y entonces tenemos una ecuación $AX=B$ donde $A$, $B$, y $X$ son matrices. Por ejemplo:

$$\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix} X = \begin{bmatrix}3 & 1 & 2\\ 5 & 7 & 2\end{bmatrix}
$$

Son tres sistemas distintos que comparten la misma matriz de coeficientes $A$.

In [8]:
a = np.array([[1,2]
             ,[3,4]])
b = np.array([[3,1,2]
             ,[5,7,2]])

In [9]:
x = la.solve(a,b)
x

array([[-1.,  5., -2.],
       [ 2., -2.,  2.]])

In [10]:
a @ x

array([[ 3.,  1.,  2.],
       [ 5.,  7.,  2.]])

Se puede precomputar una estructura que permite resolver rápidamente cualquier lado derecho.

### número de soluciones

El sistema $AX=B$ tendrá solución única si $A$ tiene rango completo. Si hay más ecuaciones (independientes) que incógnitas el sistema será incompatible, y si hay menos, tendrá múltiples soluciones.

### sistema homogéneo

Si $B = 0$ tenemos un sistema *homogéneo*: buscamos un vector $X \neq \vec 0$ que consiga $AX=\vec 0$. La solución será un elemento del *espacio nulo* de $A$. Para que exista una solución no trivial la matriz $A$ no pude ser de rango completo.

Por ejemplo, resolvamos

$$\begin{bmatrix}1 & 2 & 3 \\ 3 & 4 & 7\end{bmatrix} X = \begin{bmatrix}0 \\ 0 \end{bmatrix}
$$

In [11]:
a = np.array([[1,2,3],
              [3,-4,7]])

def null1(M):
    u,s,vt = la.svd(M)
    return vt[-1]

x = null1(a)
x

array([-0.93094934, -0.07161149,  0.35805744])

In [12]:
a @ x

array([  6.66133815e-16,   4.44089210e-16])

### sistemas sobredeterminados

En muchas aplicaciones prácticas surgen sistemas de ecuaciones que tienen más ecuaciones que incógnitas. Esto suele ocurrir cuando se intenta estimar los parámetros de un modelo a partir de un gran número de datos experimentales. Estrictamente hablando esos sistemas son incompatibles pero en muchos casos se puede encontrar una solución aceptable que minimice el error cuadrático.

Por ejemplo, el sistema siguiente no tiene solución

$$\begin{bmatrix}1 & 2 \\ 3 & 4 \\ 6 & 7\end{bmatrix} X = \begin{bmatrix}3 \\ 5 \\ 10 \end{bmatrix}
$$

In [13]:
a = np.array([[1,2],
              [3,4],
              [6,7]])
b = np.array([3,5,10])

# x = la.solve(a,b)
# daría error, expected square matrix

Pero podemos encontrar una solución aproximada con `la.lstsq`:

In [14]:
x = la.lstsq(a,b)[0]
x

array([ 0.15789474,  1.26315789])

In [15]:
a @ x

array([ 2.68421053,  5.52631579,  9.78947368])

Esta función devuelve también el error conseguido (suma de residuos cuadráticos) e información sobre lo "bien condicionado" que está el sistema.

Se puede hacer lo mismo con la *pseudoinversa*: $A^+ \equiv (A^T A)^{-1}A^T$

In [16]:
pinv = la.inv(a.T @ a) @ a.T
pinv

array([[-1.13157895, -0.44736842,  0.57894737],
       [ 0.94736842,  0.42105263, -0.36842105]])

In [17]:
pinv @ b

array([ 0.15789474,  1.26315789])

Pero si solo vamos a usarla para multiplicar por $B$ es mejor usar directamente `la.lstsq`.

### sistemas homogéneos sobredeterminados

Finalmente, también nos encontraremos con sistemas homogéneos sobredeterminados. Por ejemplo:
    
$$\begin{bmatrix}10 & 20 & 50 \\ 10 & 21  & 49 \\ 9 & 20 & 51 \\ 11 & 19 & 50\end{bmatrix} X = \begin{bmatrix}0 \\ 0 \\ 0\\0\end{bmatrix}
$$

In [18]:
a = np.array([[10, 20, 50],
              [10, 21, 49],
              [ 9, 20, 51],
              [11, 19, 50]])

x = null1(a)
x

array([-0.78774705, -0.50132179,  0.35795398])

In [19]:
a @ x

array([-0.00620698, -0.86548276,  1.13949405, -0.29263224])

La función `null1` defina más arriba obtiene el vector de norma 1 que minimiza $||AX||$. Se basa en la descomposición SVD, una de las herramientas más útiles e importantes de las matemáticas aplicadas.

### ejercicios:

- Encuentra la recta que mejor se ajusta a una colección de puntos 2D.

- Encuentra la ecuación de una elipse que pase cerca de un conjunto de puntos 2D.

### sistemas sobredeterminados y generalización

El problema fundamental en  *machine learning* es encontrar un modelo de los ejemplos de entrenamiento que resuelva correctamente la mayoría de casos futuros no vistos previamente.

Si los ejemplos de entrenamiento son representativos del problema y sobredeterminan los parámetros libres del modelo, y aún así el proceso de aprendizaje (optimización) consigue una buena solución, cabe esperar que esta solución generalice a casos futuros. Pero si los parámetros no están sobredeterminados, es seguro que encontraremos una solución perfecta para los datos de entrenamiento, sean cuales sean las etiquetas de clase que se les asignen. Se estará realizando un aprendizaje puramente memorístico, sin ninguna garantía sobre los resultados que obtendrá en futuras instancias del problema.